In [ ]:
%matplotlib notebook

import gillespy2
from gillespy2.solvers.stochkit import StochKitSolver
from tsfresh.feature_extraction.settings import MinimalFCParameters
import numpy as np
import os

from dask.distributed import Client
os.environ["STOCHKIT_HOME"] = '/home/jovyan/StochKit'

## Define Model

In [ ]:
model_doc = gillespy2.StochMLDocument.from_file("vilar_oscillator.xml")
vilar_model = model_doc.to_model("Vilar")
vilar_model.tspan = np.linspace(0, 100, 100)

## Define simulator function

In [ ]:
def set_model_parameters(params, model):
    """ params - array, needs to have the same order as model.listOfParameters """
    for e, (pname, p) in enumerate(model.listOfParameters.items()):
        model.get_parameter(pname).set_expression(params[e])
    return model

# Here we use gillespy2 numpy solver, so performance will be quite slow for this model
def simulator(params, model):
    
    from gillespy2.solvers.numpy.ssa_solver import MaxStateReached
    from sciope.stochmet.stochmet import EventFired
    
    model_update = set_model_parameters(params, model)
    num_trajectories = 1 #TODO: howto handle ensembles

    res = model_update.run(solver=StochKitSolver, show_labels=False, number_of_trajectories=num_trajectories)
    tot_res = res[0][:,1:] #should not contain timepoints
    
    return np.array(tot_res)

## Define parameter sampler

In [ ]:
default_param = np.array(list(vilar_model.listOfParameters.items()))[:,1]
bound = []
for exp in default_param:
    bound.append(float(exp.expression))
    
bound = np.array(bound)

class Sampler():
    
    def __init__(self, min_, max_):
        self.name = 'Uniform'
        self.min_ = min_
        self.max_ = max_
        self.dim = len(max_)
    
    def generate(self, n_points):
        points = np.random.uniform(self.min_, self.max_, (n_points, self.dim))
        return points

sampler = Sampler(min_=bound*0.1, max_=bound*10)

## Start local cluster using dask client

In [ ]:
c = Client()
c

## Initiate StochMET

In [ ]:
from sciope.stochmet.stochmet import StochMET

#Arg: "model" enables us the change which model to be used
simulator2 = lambda x : simulator(x, model=vilar_model)

#lets use this set of features:
default_fc_params = {'mean': None,
                     'variance': None,
                     'skewness': None,
     'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 5},
 {'f_agg': 'median', 'maxlag': 5},
 {'f_agg': 'var', 'maxlag': 5}]}


met = StochMET(simulator=simulator2, sampler=sampler, features=default_fc_params)

## Run parameter sweep (will persist and running in background)

In [ ]:
print(vilar_model.listOfSpecies.keys())

#Lets extract features for only 'Ma', 'Mr', 'C', 'A' and 'R'
idx_species = [2,5,6,7,8]

In [ ]:
met.compute(n_points=100, n_species=idx_species)

## Explore the result
Here we will explore parameter points expressed in feature space using a dimension reduction method. User can interact with points and label points according to different model behavior

In [ ]:
#First lets add some appropiate information about the model and features for interative purposes
met.data.configurations['listOfParameters'] = list(vilar_model.listOfParameters.keys())
met.data.configurations['listOfSpecies'] = list(vilar_model.listOfSpecies.keys())
met.data.configurations['listOfSummaries'] = met.summaries.features
met.data.configurations['timepoints'] = vilar_model.tspan

In [ ]:
# Here we use UMAP for dimension reduction and collect the data from persited storage
met.explore(dr_method='umap', from_distributed=True)

## Once a few points has been added we can use Semi-supervised learning

In [ ]:
from sciope.models.label_propagation import LPModel
#here lets use the dimension reduction embedding as input data
data = met.dr_model.embedding_

model_lp = LPModel()
#train using basinhopping
model_lp.train(data, met.data.user_labels, min_=0.01, max_=10, niter=50)

In [ ]:
# just to vislualize the result we will map the label distribution to the user_labels (will enable us to see the LP model 
# output when using "explore")

user_labels = np.copy(met.data.user_labels)
met.data.user_labels = model_lp.model.label_distributions_[:,0]

In [ ]:
met.explore(dr_method='umap', from_distributed=False)

In [ ]:
met.data.user_labels = user_labels